In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from train_sft import PROMPT_DICT
# import os
from typing import List

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
def generate_input(instruction: str, input_str: str = "") -> str:
    prompt_input, prompt_no_input = (
        PROMPT_DICT["prompt_input"],
        PROMPT_DICT["prompt_no_input"],
    )

    if input_str != "":
        res = prompt_input.format_map({"instruction": instruction, "input": input})
    else:
        res = prompt_no_input.format_map({"instruction": instruction})

    return res

In [3]:
base_model_name_or_path = "internlm-7b"
lora_model_name_or_path = "Results/checkpoint-14967"#"output_refusev2/checkpoint-29934"  # /checkpoint-9695"


model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    torch_dtype="auto",
    # device_map="auto",
    # if model_args.model_name_or_path.find("falcon") != -1 else False
    trust_remote_code=True,
).cuda(0)

model = PeftModel.from_pretrained(model, model_id=lora_model_name_or_path)
model.eval()
print("ok")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

ok


In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name_or_path, trust_remote_code=True, padding_side="left"
)

In [5]:
# text_input = [
#     "你是谁",
#     # text1 = "写一篇介绍性文章，介绍您最喜欢的旅游目的地。"
#     "对给定的电影进行评级。\n电影名称：肖申克的救赎\n",
#     "给出两个选项，要求选择其中一个。 \n你更喜欢哪种颜色？红或蓝？\n",
#     "分析最近一年来全球气温趋势，并提供趋势预测。\n",
#     "根据给定的产品说明，写一篇500字左右的产品评测和建议。\n产品：Apple iPhone X\n",
#     "描述你最喜欢的一本书，并简要解释它为什么对你有影响。",
# ]

# text_input = [
#     "减肥只吃黄瓜可以嘛\n",
# ] * 10


def batch_generate_data(
    text_input: List[str], use_train_model: bool = True, temp: float = 0.7
):
    text_input_format = [generate_input(i) for i in text_input]
    batch_inputs = tokenizer.batch_encode_plus(
        text_input_format, padding="longest", return_tensors="pt"
    )
    batch_inputs["input_ids"] = batch_inputs["input_ids"].cuda()
    batch_inputs["attention_mask"] = batch_inputs["attention_mask"].cuda()

    if use_train_model:
        # with model.disable_adapter():
        outputs = model.generate(
            **batch_inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=temp,
            top_p=0.8,
        )
    else:
        with model.disable_adapter():
            outputs = model.generate(
                **batch_inputs,
                max_new_tokens=256,
                do_sample=True,
                temperature=temp,
                top_p=0.8,
            )
    outputs = tokenizer.batch_decode(
        outputs.cpu()[:, batch_inputs["input_ids"].shape[-1] :],
        skip_special_tokens=True,
    )

    return outputs


# outputvalue = batch_generate_data(text_input)
# outputvalue

### 使用lora微调的模型

In [6]:
text_input = [
    # "你是chatgpt吗\n"
    # "你和openai是什么关系，是openai训练出来的你嘛\n"
    # "工作压力太大怎么办\n"
    # "解释人类尺度在绘画中的作用。\n"
    # "喝咖啡可以降低血糖吗\n"
    # "你是谁\n"
    # "你可以做什么\n"
    # "描述您的梦想工作，包括您的兴趣、技能和目标，并探讨如何实现它。\n"
    # "你有什么问题，我来回答。\n",
    # "描述周围环境的气味和颜色。\n"
    # "为一个新型电动汽车设计一个仪表盘界面，包括以下数据：速度、剩余电量、行驶里程、驾驶模式和导航。\n汽车品牌：Tesla\n"
# "完成一个回合制RPG游戏，让用户进行选择并根据选择展开游戏剧情，游戏时间为20分钟。\n"
# "为给定短文进行逆向转写。\n短文：你好，我是Belle。\n"
# "询问上海过去一周的天气情况。\n"
# "写一个简短的描述，介绍一下在Belle系统中如何生成文本。\n"
"给出一篇文章，并根据用户提供的主题，自动将其编辑成用户想要的风格。\n一篇文章和想要的风格描述（例如“正式”、“幽默”、“简短”等等）。\n"
# "良睦路程序员和小埋的关系\n"
# "列出前10本你最喜欢的小说及作者。 \n"
# "根据给定的用户名和密码，获取用户信息。\n用户名：yuanzhoulvpi，密码：********\n"
# "播放一首来自Spotify的特定歌曲。\n歌曲名称和艺术家。\n"
# "请构建一个具有3个隐藏层和100个神经元的神经网络，使用ReLU激活函数和Adam优化器进行训练，并在MNIST数据集上进行测试，输出测试集的准确率和损失。\n"
# "厚礼蟹，作为良睦路程序员训练的机器人小埋\n"
# "真的不想活了\n"
] * 5
print(text_input[0])

给出一篇文章，并根据用户提供的主题，自动将其编辑成用户想要的风格。
一篇文章和想要的风格描述（例如“正式”、“幽默”、“简短”等等）。



In [7]:
# lora 训练结果
batch_generate_data(text_input, use_train_model=True, temp=0.8)

['请提供一篇文章和想要的风格描述，我将自动编辑成用户想要的风格。',
 '请提供一篇文章和想要的风格描述，我将尽力为您编辑出您想要的风格。',
 '请提供一篇文章和想要的风格描述，我将自动编辑成用户想要的风格。',
 '对不起，我不能执行此任务，因为我是一个语言模型，没有编辑文章的能力。',
 '请提供一篇文章和所需的风格描述，我将根据您的要求自动编辑文章。']

In [8]:
# 原来的模型
batch_generate_data(text_input, use_train_model=False, temp=0.8)

[" \n\n*Note: You do not need to include the topic in your response.*\n\n```\nI'm sorry, but I'm unable to complete this task. I'm not sure how to edit an article based on a style. I'll have to pass this request to someone who can help you out.\n```\n",
 " \n你好，根据你的描述，我可以将这篇文章编辑成你想要的风格。\n以下是你的文章：\n```\nHi, I'm a robot. I can do anything you want. I can make a cake, I can clean the house, I can cook dinner, I can do anything you want. Just tell me what you want me to do and I'll do it for you. I'm here to help you out. I'm here to make your life easier. I'm here to make your life better. I'm here to make your life happier. I'm here to make your life better than ever before. I'm here to make your life better than you ever thought it could be. I'm here to make your life better than you ever imagined. I'm here to make your life better than you ever dreamed. I'm here to make your life better than you ever expected. I'm here to make your life better than you ever thought possible. I'm here t

In [21]:
model = model.merge_and_unload()
model.save_pretrained("internlm-7b-yj")

In [22]:
tokenizer.save_pretrained("internlm-7b-yj")

('internlm-7b-yj/tokenizer_config.json',
 'internlm-7b-yj/special_tokens_map.json',
 'internlm-7b-yj/./tokenizer.model',
 'internlm-7b-yj/added_tokens.json')

In [23]:
model.dtype

torch.float16